# Simple 1D Richards' Equation Model

This is a 1D Richards' equation model, based on the formulation from [Andrew Ireson](https://gmd.copernicus.org/articles/16/659/2023/gmd-16-659-2023.pdf). This model is packaged as `richards_model` class with several built in functions and plotting tools. Using a class-based approach makes it easier to assign model variables to class objects and solve the model using a scipy [ODE solver](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html). 


In [ ]:
# Import plotting libraries
from matplotlib import pyplot as plt
# package for cool and accessible colormaps (https://www.fabiocrameri.ch/colourmaps-userguide/)
# from cmcrameri import cm
import matplotlib.cm as cm 
# Import numpy
import numpy as np

# Import ODE solvers
from scipy.interpolate import interp1d
from scipy.integrate import odeint

import time
import warnings

In [ ]:
#### Define richards_model class ############### DO NOT EDIT UNLESS YOU WANT TO EDIT FUNCTIONALITY ############################
class richards_model:
    # Class attribute
    richards_model = "Define model input and output for rapid model construction and analysis"

    def __init__(self, p, dz, depth, dt, sim_time=1.0): # the default is for the rectangular profile columns
        # material properties
        self.p = p
        # Upper boundary condition
        # Option for fixed flux, default is zero
        self.qTop = 0
        # option for fixed head
        self.psiTop = []
        
        # Lower boundary condition
        # default is free drainage
        self.qBot = []
        # cross-sectional area
        self.psiBot = []

        # define grid in space
        self.dz = dz
        self.z=np.arange(self.dz/2.0, depth, self.dz)
        self.n=self.z.size
        # Grid in time 
        self.t = np.arange(0, sim_time, dt) # days

        # Initial conditions
        self.psi0=-self.z
    
    # String representation of class object
    def __str__(self):
        return f"This Richards model has the following material properties: {self.p}"

    def print_attributes_and_methods(self):
        return print([attr for attr in dir(self) if not attr.startswith('__')])
        
    # function to solve Richards equation given some model parameters
    def solve(self):
        # Solve RE model 
        start_time = time.time()  # Start the timer
        psi = odeint(RE_model, self.psi0, self.t, args=(self.dz, self.n, self.t.max(), self.p, 
                                self.qTop, self.qBot, self.psiTop, self.psiBot), mxstep=5000000)
        # Write output to model
        self.psi = psi
        # Get water content
        self.theta=thetaFun(self.psi, self.p)
        
        end_time = time.time()
        print(f"Model run successfully in {end_time - start_time:.4f} seconds")
        
    # Post process model output to get useful information
    def extract_output(self):
        if not hasattr(self, 'psi'):
            # raise warning
            warnings.warn("Warning: 'psi' is not defined. Ensure that 'psi' is initialized by solving the model before calling this function.", stacklevel=2)
            return None
        
        # Get total profile storage
        self.S = self.theta.sum(axis=1)* self.dz
        
        # Get change in storage [dVol]
        self.dS=np.zeros(self.S.size)
        self.dS[1:]=np.diff(self.S)/(np.diff(self.t))
        
        # Get infiltration flux
        if self.qTop == []:
            KTop=KFun(np.zeros(1)+ self.psiTop, self.p)
            self.qI=-KTop*((self.psiTop- self.psi[:, self.n-1])/ self.dz*2+1)
        else:
            if callable(self.qTop):  
                self.qI = self.qTop(self.t)  # If self.qTop is a function, call it with self.t  
            else:  
                self.qI = np.zeros(self.t.size)+ self.qTop  # If self.qTop is a scalar, use it directly 
  
            
        # Get discharge flux
        if self.qBot == []:
            if self.psiBot == []:
                # Free drainage
                KBot=KFun(self.psi[:,0], self.p)
                self.qD=-KBot
            else:
                # Type 1 boundary
                KBot=KFun(np.zeros(1)+ self.psiBot, self.p)
                self.qD=-KBot*((self.psi[:,0]- self.psiBot)/self.dz*2+1.0)
        else:
            self.qD=np.zeros(self.t.size)+ self.qBot

    # Post process model output to get useful information
    def plot_fluxes(self, ymin=0, ymax=1, figsize=(6, 4), dpi=150):
        plt.figure(figsize=figsize, dpi=dpi)
        plt.plot(self.t, self.dS,label='Change in storage')
        plt.plot(self.t,-self.qI,label='Infiltration')
        plt.plot(self.t,-self.qD,label='Discharge')
        plt.xlabel('Time [days]')
        plt.ylabel('Flux [m/day]')
        plt.legend()
        plt.ylim(ymin, ymax)
        plt.show()

    def plot_wc_profiles(self, freq =1, figsize=(4, 5), dpi=150):
        # Create colormap
        colors = cm.Blues(np.linspace(0.3, 1, len(self.t)))  # 'Blues' colormap

        # Plot vertical profiles
        plt.figure(figsize=figsize, dpi=dpi)
        for i in range(0, len(self.t), freq):
            plt.plot(self.theta[i,:], self.z, color=colors[i], label=f't={self.t[i]} days')

        plt.xlim(self.p['thetaR']-0.01, self.p['thetaS']+0.01)
        plt.xlabel(r'$\theta$ [-]')
        plt.ylabel('Elevation [m]')
        plt.legend()
        plt.show()

In [ ]:
# van Genuchten equation to calculate theta from head/matric potential (equation 26 and 27(
def thetaFun(psi, pars):
    Se = np.where(psi >= 0, 1.0, (1+abs(psi*pars['alpha'])**pars['n'])**(-pars['m']))  # Replace "..." with logic for psi < 0
    return pars['thetaR']+(pars['thetaS']-pars['thetaR'])*Se

# van Genuchten equation to calculate unsaturated K from head/matric potential (equation 26 and 29)
def KFun(psi,pars):
    # calculate effective saturation
    Se = np.where(psi >= 0, 1.0, (1+abs(psi*pars['alpha'])**pars['n'])**(-pars['m']))  
    return pars['Ks']*Se**pars['neta']*(1-(1-Se**(1/pars['m']))**pars['m'])**2
    
# equation 5
def CFun(psi,pars):
    # calculate effective saturation where there is a negative matric potential (i.e. unsaturated conditions
    Se = np.where(psi >= 0, 1.0, (1+abs(psi*pars['alpha'])**pars['n'])**(-pars['m']))

    dSedh=pars['alpha']*pars['m']/(1-pars['m'])*Se**(1/pars['m'])*(1-Se**(1/pars['m']))**pars['m']
    return Se*pars['Ss']+(pars['thetaS']-pars['thetaR'])*dSedh


def plot_soil_curves(pars, max_head = 10, min_head = 0):
    psi=np.linspace(-max_head, -min_head,200)
    plt.figure(figsize=(4, 10), dpi=150)
    plt.subplot(3,1,1)
    plt.plot(thetaFun(psi,pars), -psi)
    plt.xlabel(r'$\theta [-]$')
    plt.ylabel(r'Capillary Pressure Head [m]')
    plt.subplot(3,1,2)
    plt.plot(thetaFun(psi,pars), KFun(psi,pars))
    plt.xlabel(r'$\theta$ [-]')
    plt.ylabel(r'$K$ [m/d]')
    plt.subplot(3,1,3)
    plt.plot(psi, CFun(psi,pars))
    plt.ylabel(r'$C(\psi)$ [1/m]')
    plt.xlabel(r'$\psi$ [m]')
    plt.show()

In [ ]:
# Richards equation solver
# This is a function that calculated the right hand side of Richards' equation. You
# will not need to modify this function, unless you are doing something advanced. 
# This block of code must be executed so that the function can be later called.

def RE_model(psi,t,dz,n,tmax,p,qTfun,qBot,psiTop,psiBot):
       
    # Basic properties:
    C=CFun(psi,p)
   
    # initialize vectors:
    q=np.zeros(n+1)
    
    # Upper boundary
    if psiTop == []: # if no ponding, define upper flux that may vary with time
        if callable(qTfun):  # Check if qTfun is a function
            if t>tmax:
                q[n]=qTfun(tmax)
            else:
                q[n]=qTfun(t)
        else:  # If qTfun is a scalar, i.e. the flux is constant in time then set it to a fixed value
            q[n] = qTfun  
    else: # define constant head (typically a positive value)
        KTop=KFun(np.zeros(1)+psiTop,p)
        q[n]=-KTop[0]*((psiTop-psi[n-1])/dz*2+1)
    
    # Lower boundary
    if qBot == []:
        if psiBot == []:
            # Free drainage
            KBot=KFun(np.zeros(1)+psi[0],p)
            q[0]=-KBot[0]
        else:
            # Type 1 boundary
            KBot=KFun(np.zeros(1)+psiBot,p)
            q[0]=-KBot[0]*((psi[0]-psiBot)/dz*2+1.0)    
    else:
        # Type 2 boundary
        q[0]=qBot[0]
    
    # Internal nodes
    i=np.arange(0,n-1)
    Knodes=KFun(psi,p)
    Kmid=(Knodes[i+1]+Knodes[i])/2.0
    
    j=np.arange(1,n)
    q[j]=-Kmid*((psi[i+1]-psi[i])/dz+1.0)
    
    # Continuity
    i=np.arange(0,n)
    dpsidt=(-(q[i+1]-q[i])/dz)/C
    
    return dpsidt

In [ ]:
# Select which soil properties to use
# Note that to use these pre-assigned soil moisture properties you need to import the soil_params library
# import soil_params as sp
# p=sp.HygieneSandstone()
# p=sp.TouchetSiltLoam()
# p=sp.SiltLoamGE3()
# p=sp.GuelphLoamDrying()
# p=sp.GuelphLoamWetting()
# p=sp.BeitNetofaClay()

# alternatively set the soil properties in a dictionary as follows
p={}
p['thetaR']=0.1 # residual water content
p['thetaS']=0.35 # saturated water content = porosity
p['alpha']= 2.0 # 1/capillary entry pressure [m]
p['n']=5.4 # van G shape parameter
p['m']=1-1/p['n'] # second van G shape parameter
p['Ks']= 100 # saturated hydraulic conductivity [m/d] 
p['neta']=0.5 # relative permeability exponent 
p['Ss']=0.000001 # specific storage [1/m]

max_pc = 5
plot_soil_curves(p, max_pc)

### Define an instance or object of type richards model class provided the following attributes

`p`: the material properties

`dz` : grid spacing (meters)

`depth`: depth of the soil column (meters)

`dt`: time step (days)

`sim_time`: simulation time (days)

The default boundary conditions are zero flux at the top and free drainage at the bottom. The default initial conditions are full saturated at the bottom and water content throughout the column determined by the capillary characteristic curve.

### Define and run default model

In [ ]:
dz = 0.06
depth = 4 # [m]
dt = 0.5 # [days]
sim_time = 5 # [days]

free_drainage_model = richards_model(p, dz, depth, dt, sim_time)
print(free_drainage_model)

In [ ]:
free_drainage_model.solve()

In [ ]:
print(free_drainage_model.print_attributes_and_methods())

This just printed all of the model attributes and methods. Many of the attributes define different model input that can be changed to run different types of models. In addition to what is described in the introduction of the section, we can edit different model boundary conditions or initial conditions. We can also extract and print model output such as the following.

In [ ]:
free_drainage_model.extract_output()
# print model times
print(free_drainage_model.t)
# print model fluxes into the top of the model
print(free_drainage_model.qI)

In [ ]:
# set frequency of lines, frequency of 1 = plot every timestep, frequency of 10 means plot every 10 timesteps
free_drainage_model.plot_wc_profiles(freq=2)

### Now run transient flux model

In [ ]:
dz = 0.06
depth = 4 # [m]
dt = 0.1 # [days]
sim_time = 10 # [days]

# initiate model object
transient_model = richards_model(p, dz, depth, dt, sim_time)
print(transient_model)
# Ensure flux aligns with provided timesteps transient infiltration flux
qT=np.zeros(len(transient_model.t))
# set day 2 to constant flux
qT[19:29]=-2/100 # 2 cm/day

# set day 6 to constant flux
qT[59:69]=-4/100 # 4 cm/day
transient_model.qTop=interp1d(transient_model.t,qT)

# Also set initial conditions to be at residual saturation conditions
transient_model.psi0=np.ones(transient_model.n)*-5
# Alternatively, comment the line above and uncomment the line below to run a constant head lower boundary
# transient_model.psiBot = 0


In [ ]:
transient_model.solve()
transient_model.extract_output()
# plot fluxes and set ymin = -0.04 and ymax = 0.06
transient_model.plot_fluxes(-0.04, 0.06)

In [ ]:
# set frequency of lines, frequency of 1 = plot every timestep, frequency of 10 means plot every 10 timesteps
transient_model.plot_wc_profiles(freq=10)

### Example of how we update a material parameter and rerun the model

In [ ]:
# update saturated hydraulic conductivity
transient_model.p['Ks']=500
print(transient_model)

transient_model.solve()
# set frequency of lines, frequency of 1 = plot every timestep, frequency of 10 means plot every 10 timesteps
transient_model.plot_wc_profiles(freq=10)

What parameter was changed between these last two models? How does changing this parameter change the model results? What does this imply about trying to measure recharge in sandy or coarser soils using measurements of water content?

## Activity
Now build a model where you start with fully saturated conditions and the bottom of the model represents the water table. Run the model until equilibrium drainage conditions are reached. Use a different material for the model. Note that if you want to import pre-assigned materials then you need to import the `soil_params` library. Alternatively, you can define your own soil properties as we did above.

In [ ]:
# Import a library that contains soil moisture properties and functions
import soil_params as sp

In [ ]:
# initiate model object
drainage_model = richards_model(sp.HygieneSandstone(), 0.06, 4, 0.5, 5)
# Print user-defined attributes and methods
print(drainage_model.print_attributes_and_methods())

This provides a list of all the model variables and methods (built in functions). Here are the boundary condition options:

In [ ]:
print(drainage_model.qTop)
print(drainage_model.qBot)
print(drainage_model.psiTop)
print(drainage_model.psiBot)

Here are the initial conditions settings

In [ ]:
print(drainage_model.psi0)

In [ ]:
# update model initial and boundary conditions to start fully saturated and run the model until equilibrium drainage conditions are reached with no recharge flux

drainage_model.solve()

In [ ]:
# set frequency of lines, frequency of 1 = plot every timestep, frequency of 10 means plot every 10 timesteps
drainage_model.plot_wc_profiles(freq=1)

## Other plotting options
Note that `z` increases in the upward direction.

In [ ]:
# Create colormap
colors = cm.cividis(np.linspace(0.3, 1, len(transient_model.t)))  # 'Blues' colormap
interval = 10
# Plot time series of different cells
plt.figure(figsize=(6, 3), dpi=150)
for i in range(0, transient_model.n, interval):
    plt.plot(transient_model.t,-transient_model.psi[:,i], color=colors[i], label=f'z={transient_model.z[i]:.1f} [m]')

plt.xlabel('Time [days]')
plt.ylabel('Capillary pressure [m]')
plt.legend()
plt.show()